In [1]:
import pandas as pd 
import re

def format_learning_survey(level):
    excel_file = '2020 '+level + ' Subject Class List.xlsx'
    
    if level in ('SH1', 'SH2'):
        df = pd.read_excel(excel_file, sheet_name = 'Subject Class Lists')
        columns_to_delete = ['New Reg No', 'Semester']
        df["Subject Code1"] = df["Subject Code"].str.slice(stop = 2)
        df.rename(columns = {'StudentName': 'Student Name', 'TeacherName': 'Teacher Name'}, inplace = True)
    else:
        df = pd.read_excel(excel_file, sheet_name = level)
        if level == 'JH1' or level == 'JH2':
            sem = ['Both', 2]
            df = df[df.Semester.isin(sem)]
        columns_to_delete = ['Reg No.', 'Semester'] #, 'Subject Class' not deleted for the moment
        df["Subject Code"] = df["Subject Code"].str.slice(start = 1)
        if level == 'JH3':
            df.rename(columns = {'StudentName': 'Student Name', 'TeacherName': 'Teacher Name', 'Subject Code': 'Subject Code1'}, inplace = True)
        else:
            df.rename(columns = {'Subject Code': 'Subject Code1'}, inplace = True)
    
    df['Teacher Name'] = df['Teacher Name'].str.upper()
    df['Teacher Name'] = df['Teacher Name'].str.strip()
    df.drop(columns_to_delete, axis = 1, inplace = True)

    lookup_df1 = pd.read_excel('Subject Code.xlsx', sheet_name='Sheet1')
    subject_name_df = pd.DataFrame(lookup_df1, columns = ['Subject Code1', 'Subject Name'])
    df = df.merge(subject_name_df, on='Subject Code1', how='left')

    teacher_id_df = pd.DataFrame(lookup_df1, columns = ['Teacher Name', 'UserID'])
    teacher_id_df['Teacher Name'] = teacher_id_df['Teacher Name'].str.upper()
    teacher_id_df['Teacher Name'] = teacher_id_df['Teacher Name'].str.strip()

    df = df.merge(teacher_id_df, on='Teacher Name', how='left')
    df.rename(columns = {'UserID': 'TeacherIDs'}, inplace = True)

    lookup_df2 = pd.read_excel('2020_SLS_userid.xlsx', sheet_name = level)
    lookup_df2.rename(columns = {'Name': 'Student Name'}, inplace = True)
    SLS_id_df = pd.DataFrame(lookup_df2, columns = ['Student Name', 'Userid'])

    df = df.merge(SLS_id_df, on='Student Name', how='left')
    # cols = list(df_jh1.columns.values)
    # print(cols)

    if level in ('SH1', 'SH2'):
        df = df[['Userid', 'Class', 'Student Name', 'Gender', 'Subject Code', 'Subject Code1', 'Subject Name', 'Subject Class', 'Teacher Name', 'TeacherIDs']]
    else:    
        df = df[['Userid', 'PM Class', 'Student Name', 'Gender', 'Subject Code1', 'Subject Name', 'Subject Class', 'Teacher Name', 'TeacherIDs']]

    return df

In [2]:
import pandas as pd 
import re

levels = ['JH1', 'JH2', 'JH3', 'JH4', 'SH1', 'SH2']

# df_jh1.to_excel('JH1.xlsx', index = False)

df_teach = pd.read_excel('TeachersIDs.xlsx', sheet_name = 'TeachersIDs')

with pd.ExcelWriter('2020 Subject Class List_all levels_T&L Survey.xlsx') as writer:
    df_teach.to_excel(writer, sheet_name='TeachersIDs', index = False)
    for level in levels:
        df = format_learning_survey(level)
        df.style.highlight_null(null_color = 'red').to_excel(writer, sheet_name=level, index = False)

In [9]:
import pandas as pd 
import re

df_jh1 = pd.read_excel('Sample_students_data.xlsx',
                        sheet_name='Sample JH Data')

df_jh1["Subject Code"] = df_jh1["Subject Code"].str.slice(start = 1)

display(df_jh1.head())


,NRIC,Reg No.,PM Class,Student Name,Gender,Semester,Subject Code,Subject Class,Teacher Name
0,NaN,2001001,JH101,ANDREA ANG HUI SHAN,F,1,GA,NaN,YEO CHIN KENT ERNEST
1,NaN,2001001,JH101,ANDREA ANG HUI SHAN,F,Both,CL,1CL1,LOW BOON CHU
2,NaN,2001001,JH101,ANDREA ANG HUI SHAN,F,Both,GE,NaN,"KOK JING YAO, JULIANA"
3,NaN,2001001,JH101,ANDREA ANG HUI SHAN,F,Both,HI,NaN,MAK WEISHAN
4,NaN,2001001,JH101,ANDREA ANG HUI SHAN,F,Both,IS,NaN,TEO TAI WEI


In [ ]:
# df_jh1 = pd.read_excel('Sample_students_data.xlsx',
#                         sheet_name='Sample JH Data')

# columns_to_delete = ['NRIC', 'Semester', 'Subject Class']
# df_jh1.drop(columns_to_delete, axis = 1, inplace = True)

# df_jh1["Subject Code"] = df_jh1["Subject Code"].str.slice(start = 1)

# df_jh1.rename(columns = {'Subject Code': 'Subject Code1', 'Teacher Name': 'Teacher Names'}, inplace = True)

# lookup_df = pd.read_excel('Subject Code.xlsx', sheet_name='Sheet1')
# subject_name_df = pd.DataFrame(lookup_df, columns = ['Subject Code1', 'Subject Name'])
# df_jh1 = df_jh1.merge(subject_name_df, on='Subject Code1', how='left')

# teacher_id_df = pd.DataFrame(lookup_df, columns = ['Teacher Names', 'UserID'])
# df_jh1 = df_jh1.merge(teacher_id_df, on='Teacher Names', how='left')

# df_jh1 = df_jh1[['Reg No.', 'PM Class', 'Student Name', 'Gender', 'Subject Code1', 'Subject Name', 'Teacher Names', 'UserID']]



In [1]:
class teacher:
    def __init__(self, name, sls_id):
        self._name = name
        self._sls_id = sls_id

    @property
    def name(self):
        return self._name

    @property
    def sls_id(self):
        return self._sls_id


In [38]:


# print(teacher_teams)


In [39]:


    # print(teacher_dict)

['CHAN PUI SHAN SELINA', 'SEE YEOW HOE']


In [43]:
import csv

with open('Missing Teacher Pairs.csv') as f:
    reader = csv.reader(f)
    header = next(reader)
    teacher_teams = [row[0] for row in reader if row[0] != '']
    
with open('TeachersIDs.csv') as f:
    reader = csv.reader(f)
    header = next(reader)
    teacher_dict = {}
    for row in reader:
        teacher_dict[row[0]] = row[1]

with open('output.csv', 'w', newline='') as file:
    writer=csv.writer(file)
    for team in teacher_teams:
        uncleaned_names = team.split(',')
        teacher_search = []
        for t in uncleaned_names:
            teacher_search.append(t.strip())
        #print(teacher_search)
        uid = [teacher_dict[i] for i in teacher_search]
        teacher_ids = ', '.join(uid)
        name_id = []
        name_id.append(team)
        name_id.append(teacher_ids)
        print(name_id)
        writer.writerow(name_id)

['CHAN PUI SHAN SELINA, SEE YEOW HOE', 'MOE-76265A, MOE-25794F']
['TEO TAI WEI, GAYLE SIM', 'MOE-16867Z, MOE-02617E']
['TEO CHOR HOWE, LIM HUI CHI', 'MOE-45059G, MOE-43432B']
['ARTINA BINTE SELAMAT, ZULIANA MOHD SUHOD', 'MOE-40573I, Contract teacher ']
['LUM ZHI YONG AARON, CHAI YEN FONG', 'MOE-28034A, MOE-37319N']
['TANG AILEEN, LUM ZHI YONG AARON', 'MOE-75448H, MOE-28034A']
['FONG SU FERN VIVIEN, WANG MURAN', 'MOE-12019B, MOE-82609B']
['CHIN KAI QING JULIANA, PHUA WILLIAM', 'MOE-5098A, MOE-07041I']
['LYE WAI LENG, TAN XUAN WEN JEFFREY', 'MOE-48895D, MOE-31263J']
['LAM YUEN WAI, LIM HUI CHI', 'MOE-06485I, MOE-43432B']
['SING NIGEL JON, YEO CHIN KENT ERNEST', 'MOE-03232F, MOE-16360J']
['ZULIANA MOHD SUHOD, IRYIANNA AHMAD', 'Contract teacher , MOE-09854I']
['KWEK YUAN CHIA, IRYIANNA AHMAD', 'MOE-24571I, MOE-09854I']
['GAYLE SIM, TANG AILEEN, LUM ZHI YONG AARON', 'MOE-02617E, MOE-75448H, MOE-28034A']
['LIM CHIN MIN, PHUA WILLIAM', 'MOE-22816H, MOE-07041I']
['GUO LANHUA, LIM CHIN MIN', 'M

In [14]:
with open('output.csv', 'w', newline='') as file:
    writer=csv.writer(file)
    for team in teacher_teams:
        teacher_search = team.split(', ')
        uid = [teacher_dict[i] for i in teacher_search]
        teacher_ids = ', '.join(uid)
        name_id = []
        name_id.append(team)
        name_id.append(teacher_ids)
        writer.writerow(name_id)


KeyError: 'LIN DONGRUI JUSTIN,PRIYA V.JOTHI'

In [16]:
import csv

teacher_dict = {'Kwek Yuan Chia':'MOE-24571I', 'Taranpal Singh':'MOE-05653C'}
teacher_names = [['Kwek Yuan Chia, Taranpal Singh']]

teacher_search = teacher_names[0][0].split(', ')
uid = []
for i in teacher_search:
    uid.append(teacher_dict[i])

teacher_ids = ', '.join(uid)
# print(teacher_ids)

teacher_names[0].append(teacher_ids)
print(teacher_names[0])

with open('output.csv', 'w', newline='') as file:
    writer=csv.writer(file)
    for row in teacher_names:
        writer.writerow(row)


['Kwek Yuan Chia, Taranpal Singh', 'MOE-24571I, MOE-05653C']
